In [1]:
import fastNLP
import xml.dom.minidom

### 处理XML文件到csv文件

In [2]:
import csv
def TrainXML():
    dom = xml.dom.minidom.parse('SMP2019_ECISA_Train.xml')
    root = dom.documentElement
    f = open('train.csv','w',encoding='utf-8')
    csv_writer = csv.writer(f)
    sentences = root.getElementsByTagName('Sentence')
    for sentence in sentences:
        label = sentence.getAttribute('label')
        if label == '0':
            csv_writer.writerow(["0",sentence.firstChild.data])
            continue
        elif label == '1':
            csv_writer.writerow(["1",sentence.firstChild.data])
            continue
        elif label == '2':
            csv_writer.writerow(["2",sentence.firstChild.data])
            continue
        else:  
            # 没有label属性，无情感标注为0
            #csv_writer.writerow(["0",sentence.firstChild.data])
            continue
TrainXML()
print("part of train xml solve")

part of train xml solve


In [3]:
def DevXML():
    dom = xml.dom.minidom.parse('SMP2019_ECISA_Dev.xml')
    root = dom.documentElement
    f = open('dev.csv','w',encoding='utf-8')
    csv_writer = csv.writer(f)
    sentences = root.getElementsByTagName('Sentence')
    for sentence in sentences:
        label = sentence.getAttribute('label')
        if label == '0':
            csv_writer.writerow(["0",sentence.firstChild.data])
            continue
        elif label == '1':
            csv_writer.writerow(["1",sentence.firstChild.data])
            continue
        elif label == '2':
            csv_writer.writerow(["2",sentence.firstChild.data])
            continue
        else:  
            # 没有label属性，无情感标注为0
            #csv_writer.writerow(["0",sentence.firstChild.data])
            continue
DevXML()
print("part of devxml solve")

part of devxml solve


In [4]:
def TestXML():
    dom = xml.dom.minidom.parse('SMP2019_ECISA_Test.xml')
    root = dom.documentElement
    f = open('test.csv','w',encoding='utf-8')
    csv_writer = csv.writer(f)
    sentences = root.getElementsByTagName('Sentence')
    for sentence in sentences:
        label = sentence.getAttribute('label')
        if label == '0':
            csv_writer.writerow(["0",sentence.firstChild.data])
            continue
        elif label == '1':
            csv_writer.writerow(["1",sentence.firstChild.data])
            continue
        elif label == '2':
            csv_writer.writerow(["2",sentence.firstChild.data])
            continue
        else:  
            # 没有label属性，无情感标注为0
            #csv_writer.writerow(["0",sentence.firstChild.data])
            continue
TestXML()
print("part of testxml solve")

part of testxml solve


### load数据为databundle

In [5]:
import pandas as pd

train_data_pd = pd.read_csv(r'C:\Users\Rye\Desktop\cnn\train.csv', sep = ',')
dev_data_pd = pd.read_csv(r'C:\Users\Rye\Desktop\cnn\dev.csv', sep = ',')
test_data_pd = pd.read_csv(r'C:\Users\Rye\Desktop\cnn\test.csv', sep = ',')

train_data_pd.to_csv('train.txt',sep=',', index=False,header=None,encoding='utf-8')
dev_data_pd.to_csv('dev.txt',sep=',', index=False,header=None,encoding='utf-8')
test_data_pd.to_csv('test.txt',sep=',', index=False,header=None,encoding='utf-8')

#windows下必须转化成txt才能读入

In [6]:
from fastNLP.io.loader import CSVLoader
data_set_loader = CSVLoader(
    headers=('target','raw_words'), sep=','
)
import os
os.remove('C:/Users/Rye/Desktop/cnn/train.csv') 
os.remove('C:/Users/Rye/Desktop/cnn/test.csv') 
os.remove('C:/Users/Rye/Desktop/cnn/dev.csv') 
data_bundle = data_set_loader.load('C:/Users/Rye/Desktop/cnn')

train_data = data_bundle.get_dataset('train')
dev_data = data_bundle.get_dataset('dev')
test_data = data_bundle.get_dataset('test')

data_bundle.set_dataset(test_data, 'test')
data_bundle.set_dataset(train_data, 'train')
print(data_bundle.get_dataset('test')[:10])
print(len(train_data),len(dev_data),len(test_data))
print(data_bundle)
#懒得加代码了，必须手动删除csv文件

+--------+-------------------------------------+
| target | raw_words                           |
+--------+-------------------------------------+
| 2      | 王健林把万达广场也卖了，万达都快... |
| 2      | 【易到司机提现延期背后:或因乐视...  |
| 1      | 建行调乐视员工信用卡额度至1元排...  |
| 1      | 这是一个蛮有意思的小岛，距离香港... |
| 2      | 夜半撞车我的评分：符合自己风格的... |
| 2      | 看到北京工作的同学这时发的雾霾图... |
| 2      | #我为都教授看春晚#我宁愿为赵本山... |
| 2      | 阿姨被淘汰我没话说，但是里斯蛋泥... |
| 1      | 如果我没吃完的话……哈哈哈…吃完...    |
| 0      | 馆内餐饮：馆内有咖啡厅、面包店等... |
+--------+-------------------------------------+
14773 5142 3808
In total 3 datasets:
	dev has 5142 instances.
	test has 3808 instances.
	train has 14773 instances.



### 使用DataSet预处理文本

In [7]:
from fastNLP import DataSet
import re
def get_words(instance):
    sentence = instance['raw_words']
    # 首先分割 英文 以及英文和标点
    pattern_char_1 = re.compile(r'([\W])')
    parts = pattern_char_1.split(sentence)
    parts = [p for p in parts if len(p.strip())>0]
    # 分割中文
    pattern = re.compile(r'([\u4e00-\u9fa5])')
    words = pattern.split(sentence)
    words = [w for w in words if len(w.strip())>0]
    return words
train_data.apply(get_words, new_field_name='words')
dev_data.apply(get_words, new_field_name='words')
test_data.apply(get_words, new_field_name='words')

def get_len(instance):
    sentence = instance['words']
    return len(sentence)
train_data.apply(get_len, new_field_name='seq_len')
dev_data.apply(get_len, new_field_name='seq_len')
test_data.apply(get_len, new_field_name='seq_len')


print(data_bundle.get_dataset('test')[:10])

+--------+----------------------+---------------------+---------+
| target | raw_words            | words               | seq_len |
+--------+----------------------+---------------------+---------+
| 2      | 王健林把万达广场...  | ['王', '健', '林... | 20      |
| 2      | 【易到司机提现延...  | ['【', '易', '到... | 68      |
| 1      | 建行调乐视员工信...  | ['建', '行', '调... | 25      |
| 1      | 这是一个蛮有意思...  | ['这', '是', '一... | 29      |
| 2      | 夜半撞车我的评分...  | ['夜', '半', '撞... | 42      |
| 2      | 看到北京工作的同...  | ['看', '到', '北... | 130     |
| 2      | #我为都教授看春晚... | ['#', '我', '为'... | 83      |
| 2      | 阿姨被淘汰我没话...  | ['阿', '姨', '被... | 25      |
| 1      | 如果我没吃完的话...  | ['如', '果', '我... | 24      |
| 0      | 馆内餐饮：馆内有...  | ['馆', '内', '餐... | 44      |
+--------+----------------------+---------------------+---------+


### 使用Vocabulary转换文本

In [8]:
from fastNLP import Vocabulary
vocab = Vocabulary()
vocab.from_dataset(train_data, dev_data, test_data, field_name='words')
vocab.index_dataset(train_data, dev_data, test_data, field_name='words')
data_bundle.set_vocab(vocab, 'words')
char_vocab = data_bundle.get_vocab('words')
#char_vocab.to_index('讨')
print(char_vocab)
#print(data_bundle)

target_vocab = Vocabulary(padding=None, unknown=None)
target_vocab.from_dataset(train_data, dev_data, test_data, field_name='target')
target_vocab.index_dataset(train_data, dev_data, test_data, field_name='target')
data_bundle.set_vocab(target_vocab, 'target')
print(data_bundle)
print(data_bundle.get_dataset('test')[:10])

Vocabulary(['//@nokki', '卡', '布', '诺', '琪']...)
In total 3 datasets:
	dev has 5142 instances.
	test has 3808 instances.
	train has 14773 instances.
In total 2 vocabs:
	words has 12633 entries.
	target has 3 entries.

+--------+----------------------+---------------------+---------+
| target | raw_words            | words               | seq_len |
+--------+----------------------+---------------------+---------+
| 1      | 王健林把万达广场...  | [150, 992, 290, ... | 20      |
| 1      | 【易到司机提现延...  | [201, 718, 19, 1... | 68      |
| 2      | 建行调乐视员工信...  | [129, 68, 595, 4... | 25      |
| 2      | 这是一个蛮有意思...  | [15, 5, 8, 13, 1... | 29      |
| 1      | 夜半撞车我的评分...  | [420, 335, 1851,... | 42      |
| 1      | 看到北京工作的同...  | [21, 19, 73, 90,... | 130     |
| 1      | #我为都教授看春晚... | [462, 7, 42, 325... | 83      |
| 1      | 阿姨被淘汰我没话...  | [306, 1714, 107,... | 25      |
| 2      | 如果我没吃完的话...  | [185, 208, 7, 37... | 24      |
| 0      | 馆内餐饮：馆内有...  | [295, 267, 735, ... | 44      |
+----

### 使用Embedding模块将文本转成向量

In [9]:
import torch
from fastNLP.embeddings import StaticEmbedding

word2vec_embed = StaticEmbedding(char_vocab, model_dir_or_name='cn-char-fastnlp-100d')

Found 4820 out of 12633 words in the pre-training embedding.


..\aten\src\ATen\native\TensorFactories.cpp:361: UserWarning: Deprecation warning: In a future PyTorch release torch.full will no longer return tensors of floating dtype by default. Instead, a bool fill_value will return a tensor of torch.bool dtype, and an integral fill_value will return a tensor of torch.long dtype. Set the optional `dtype` or `out` arguments to suppress this warning.


In [10]:
from fastNLP.io import CSVLoader
from fastNLP import Vocabulary, CrossEntropyLoss, AccuracyMetric, ClassifyFPreRecMetric
from fastNLP.models import CNNText
from fastNLP import Trainer
from fastNLP import Tester

train_data.set_input('words','seq_len')
train_data.set_target('target')
dev_data.set_input('words','seq_len')
dev_data.set_target('target')
test_data.set_input('words','seq_len')
test_data.set_target('target')
test_data.print_field_meta()

+-------------+--------+-----------+-------+---------+
| field_names | target | raw_words | words | seq_len |
+-------------+--------+-----------+-------+---------+
|   is_input  | False  |   False   |  True |   True  |
|  is_target  |  True  |   False   | False |  False  |
| ignore_type | False  |           | False |  False  |
|  pad_value  |   0    |           |   0   |    0    |
+-------------+--------+-----------+-------+---------+


In [11]:
model_cnn = CNNText(word2vec_embed, num_classes=3, dropout=0)
print(model_cnn)

CNNText(
  (embed): Embedding(
    (embed): StaticEmbedding(
      (dropout_layer): Dropout(p=0, inplace=False)
      (embedding): Embedding(12633, 100, padding_idx=0)
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (conv_pool): ConvMaxpool(
    (convs): ModuleList(
      (0): Conv1d(100, 30, kernel_size=(1,), stride=(1,), bias=False)
      (1): Conv1d(100, 40, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (2): Conv1d(100, 50, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
    )
  )
  (dropout): Dropout(p=0, inplace=False)
  (fc): Linear(in_features=120, out_features=3, bias=True)
)


In [12]:

from sklearn.metrics import f1_score, classification_report
from fastNLP import MetricBase
import numpy as np

class FMetric(MetricBase):
    def __init__(self):
        super().__init__()

        # 根据你的情况自定义指标
        self.ture = []
        self.pre = []

    def evaluate(self, target, pred): # 这里的名称需要和dataset中target field与model返回的key是一样的，不然找不到对应的value
        # dev或test时，每个batch结束会调用一次该方法，需要实现如何根据每个batch累加metric
        #tp = torch.max(pred,1)[1].view(target.size()).data.cpu().numpy() #[1,3]
        #tt = target.data.cpu().numpy()
        fp = pred.data.cpu().numpy()
     #  print(pred)
     #   print(target.size())
        ft = target.data.cpu().numpy()
        tp = np.ndarray.tolist(fp)
        tt = np.ndarray.tolist(ft)
        
        self.ture.extend(tt)
        self.pre.extend(tp)
        #print(type(tp))
        #print(tp)
        #f1s = f1_score(tt, tp, average='macro' ) #计算每个batch的f1-score
        # report = classification_report(tt, tp, labels=[0,1,2])
        
        #self.total += target.size(0)
        #self.f1_count += f1s * target.size(0)


    def get_metric(self, reset=True): # 在这里定义如何计算metric
        # f1 = self.f1_count / self.total
        report = classification_report(self.ture, self.pre, labels=[0,1,2])
        if reset: # 是否清零以便重新计算
            #self.f1_count = 0
            #self.total = 0
            self.ture.clear()
            self.pre.clear()
        return {'Report': report}  # 需要返回一个dict，key为该metric的名称，该名称会显示到Trainer的progress bar中
    

In [13]:

from torch.optim import Adam
from fastNLP import CrossEntropyLoss
loss = CrossEntropyLoss()
optimizer = Adam(model_cnn.parameters(), lr=0.001)
device = 0 if torch.cuda.is_available() else 'cpu'  # 如果有gpu的话在gpu上运行，训练速度会更快
metric = FMetric()
trainer_cnn = Trainer(train_data=data_bundle.get_dataset('train'), model=model_cnn, loss=loss,
                  optimizer=optimizer, batch_size=1, dev_data=data_bundle.get_dataset('dev'),
                  metrics=metric,device=device)
trainer_cnn.train()



input fields after batch(if batch size is 1):
	words: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([1, 15]) 
	seq_len: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([1]) 
target fields after batch(if batch size is 1):
	target: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([1]) 

training epochs started 2020-07-10-15-44-17-055037


C:\Users\Rye\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Rye\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Evaluate data in 32.32 seconds!
Evaluation on dev at Epoch 1/10. Step:14773/147730: 
FMetric: Report=              precision    recall  f1-score   support

           0       0.83      0.86      0.84      2553
           1       0.78      0.66      0.72      1358
           2       0.59      0.65      0.62      1231

    accuracy                           0.76      5142
   macro avg       0.73      0.72      0.73      5142
weighted avg       0.76      0.76      0.76      5142




Evaluate data in 32.84 seconds!
Evaluation on dev at Epoch 2/10. Step:29546/147730: 
FMetric: Report=              precision    recall  f1-score   support

           0       0.84      0.84      0.84      2553
           1       0.72      0.74      0.73      1358
           2       0.63      0.60      0.61      1231

    accuracy                           0.76      5142
   macro avg       0.73      0.73      0.73      5142
weighted avg       0.76      0.76      0.76      5142




Evaluate data in 32.17 seconds!
Evaluation on dev at Epoch 3/10. Step:44319/147730: 
FMetric: Report=              precision    recall  f1-score   support

           0       0.85      0.78      0.81      2553
           1       0.65      0.77      0.71      1358
           2       0.58      0.55      0.56      1231

    accuracy                           0.72      5142
   macro avg       0.69      0.70      0.69      5142
weighted avg       0.73      0.72      0.73      5142




...

Evaluate data in 32.62 seconds!
Evaluation on dev at Epoch 7/10. Step:103411/147730: 
FMetric: Report=              precision    recall  f1-score   support

           0       0.82      0.79      0.80      2553
           1       0.72      0.61      0.66      1358
           2       0.52      0.65      0.58      1231

    accuracy                           0.71      5142
   macro avg       0.68      0.68      0.68      5142
weighted avg       0.72      0.71      0.71      5142




Evaluate data in 30.82 seconds!
Evaluation on dev at Epoch 8/10. Step:118184/147730: 
FMetric: Report=              precision    recall  f1-score   support

           0       0.81      0.82      0.81      2553
           1       0.70      0.66      0.68      1358
           2       0.57      0.59      0.58      1231

    accuracy                           0.72      5142
   macro avg       0.69      0.69      0.69      5142
weighted avg       0.72      0.72      0.72      5142




Evaluate data in 30.67 seconds!
Evaluation on dev at Epoch 9/10. Step:132957/147730: 
FMetric: Report=              precision    recall  f1-score   support

           0       0.81      0.82      0.82      2553
           1       0.68      0.72      0.70      1358
           2       0.60      0.54      0.57      1231

    accuracy                           0.73      5142
   macro avg       0.70      0.69      0.69      5142
weighted avg       0.73      0.73      0.73      5142




Evaluate data in 30.25 seconds!
Evaluation on dev at Epoch 10/10. Step:147730/147730: 
FMetric: Report=              precision    recall  f1-score   support

           0       0.81      0.80      0.80      2553
           1       0.69      0.69      0.69      1358
           2       0.58      0.58      0.58      1231

    accuracy                           0.72      5142
   macro avg       0.69      0.69      0.69      5142
weighted avg       0.72      0.72      0.72      5142


Reloaded the best model.

In Epoch:3/Step:44319, got best dev performance:
FMetric: Report=              precision    recall  f1-score   support

           0       0.85      0.78      0.81      2553
           1       0.65      0.77      0.71      1358
           2       0.58      0.55      0.56      1231

    accuracy                           0.72      5142
   macro avg       0.69      0.70      0.69      5142
weighted avg       0.73      0.72      0.73      5142



{'best_eval': {'FMetric': {'Report': '              precision    recall  f1-score   support\n\n           0       0.85      0.78      0.81      2553\n           1       0.65      0.77      0.71      1358\n           2       0.58      0.55      0.56      1231\n\n    accuracy                           0.72      5142\n   macro avg       0.69      0.70      0.69      5142\nweighted avg       0.73      0.72      0.73      5142\n'}},
 'best_epoch': 3,
 'best_step': 44319,
 'seconds': 2616.53}

In [14]:
test_cnn = Tester(data_bundle.get_dataset('test'), model=model_cnn,
                   batch_size=1,
                  metrics=metric)
test_cnn.test()

Evaluate data in 24.0 seconds!
[tester] 
FMetric: Report=              precision    recall  f1-score   support

           0       0.83      0.71      0.77      1910
           1       0.63      0.80      0.71       978
           2       0.57      0.57      0.57       920

    accuracy                           0.70      3808
   macro avg       0.68      0.69      0.68      3808
weighted avg       0.72      0.70      0.70      3808



{'FMetric': {'Report': '              precision    recall  f1-score   support\n\n           0       0.83      0.71      0.77      1910\n           1       0.63      0.80      0.71       978\n           2       0.57      0.57      0.57       920\n\n    accuracy                           0.70      3808\n   macro avg       0.68      0.69      0.68      3808\nweighted avg       0.72      0.70      0.70      3808\n'}}

In [15]:
'''
#device = 0 if torch.cuda.is_available() else 'cpu'  # 如果有gpu的话在gpu上运行，训练速度会更快
#print(device)
trainer_cnn = Trainer(model=model_cnn, train_data=train_data, dev_data=dev_data,loss=CrossEntropyLoss(), metrics=AccuracyMetric())

trainer_cnn.train()'''

"\n#device = 0 if torch.cuda.is_available() else 'cpu'  # 如果有gpu的话在gpu上运行，训练速度会更快\n#print(device)\ntrainer_cnn = Trainer(model=model_cnn, train_data=train_data, dev_data=dev_data,loss=CrossEntropyLoss(), metrics=AccuracyMetric())\n\ntrainer_cnn.train()"

In [16]:
print(trainer_cnn)